### Title

#### Project: FRES
#### Author: Katie Murenbeeld
#### Date: 30 June 2022
#### Description: 
**Work on description** This notebook works through a basic example of generating a dataframe with updated FATES parameter values for four root parameters of a single plant function type (PFT) and then generates a set of parameter files for testing or running multi-instance FATES simulations.

In [1]:
### To create and analyze (if needed) a dataframe of potential parameter values.
import pandas as pd
import numpy as np
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
import csv

sns.set_context("notebook", font_scale=1.75, rc={"lines.linewidth": 2.5, "lines.markersize": 6.0, 'lines.markeredgewidth': 1.0})

### For generating the .nc files
import netCDF4 as nc4
import sys
import os
import shutil

For this example we will be working with:
1. The default fates parameter file for a single plant functional type (PFT). Here the broadleaf_evergreen_extratrop_shrub
2. Only updating: 
    * fates_allom_zroot_max_z - the maximum rooting depth defined at dbh = fates_allom_zroot_max_dbh. note: max_z=min_z=large, sets rooting depth to soil depth

    * fates_allom_l2fr - allocation paramter: fine root C per leaf C

    * the fates_allom_fnrt_prof_a and  fates_allom_fnrt_prof_b parameters - fine root profile parameters.
3. Generate 10 parameter files with different values for each of the listed parameters 

In reality you will likely be working from slightly modified parameter file that is calibrated for the specific point or region. Additionally, you may also want multiple PFTs, for example grass and shrubs or shrubs and trees. I will include notes at the end for how to update parameter files with more than 1 PFT. 

In [2]:
## 1. Generate a data frame with columns named for the updated parameters 
## Depending on the set of experiments there may be other ways to set this value. 
## For example, randomly sampling from a normal or other type of distribution. 

root_z_max = pd.Series(np.arange(1,11,1))
l2fr = pd.Series(np.arange(0.1, 1.1, 0.1))
fnrt_prof_a = pd.Series(np.arange(5, 7, 0.2))
fnrt_prof_b = pd.Series(np.arange(1, 2, 0.1))

column_names = ['root_depth','l2fr','fnrt_prof_a', 'fnrt_prof_b']

In [50]:
df_tmp = pd.concat([root_z_max, l2fr, fnrt_prof_a, fnrt_prof_b], axis=1)
df_tmp.columns = column_names
df_tmp


,root_depth,l2fr,fnrt_prof_a,fnrt_prof_b
0,1,0.1,5.0,1.0
1,2,0.2,5.2,1.1
2,3,0.3,5.4,1.2
3,4,0.4,5.6,1.3
4,5,0.5,5.8,1.4
5,6,0.6,6.0,1.5
6,7,0.7,6.2,1.6
7,8,0.8,6.4,1.7
8,9,0.9,6.6,1.8
9,10,1.0,6.8,1.9


In [1]:
## 2. Create 10 new parameter files with the new parameter values 

# Set the file template. In this case a parameter file with only the default values for the shrub PFT.
# If you decide to have multiple PFTs make sure the template is correct.
# Note: The parameter file here is on my local. I used Globus to transfer the original file and to 
# uplaod the new parameter files. 

filename_template = '/Users/kathrynmurenbeeld/CODING/FRES/DATA/fates_US-Rms_shrub.nc'
filename_output_basename = 'param_file_fates_US-Rms_shrub_exp1_'

# Set the index value for each parameter
# Because we are only updating 4 parameters this may not be necessary
rootdepth_indx = 0
l2fr_indx = 1
fnrtprofa_indx = 2
fnrtprofb_indx = 3

In [8]:
print(df_tmp.shape)

## This function will print out whether the new file as named already exists and will be over written 
## and if the file does not yet exist and will be created.

def copy_clobber(filename1, filename2):
    try:
        os.remove(filename2)
        print('replacing file: '+filename2)
    except:
        print('file does not yet exist, creating: '+filename2)
    shutil.copyfile(filename1, filename2)

(10, 4)


In [36]:
for i in range(10): #shape or length of columns in dataframe. In this example 10
    filename_out = filename_output_basename+str(i+1).zfill(4)+'_c220630.nc' #yymmdd
    copy_clobber(filename_template,filename_out)
    fin = nc4.Dataset(filename_out, 'r+')
    
    # Replace the default fates_turnover_fnrt value with a value from the dataframe.
    rootdepthvar = fin.variables['fates_allom_zroot_max_z']
    rootdepthvar[:] = df_tmp.iloc[i,rootdepth_indx].squeeze() # for 1 PFT
    #rootlongvar[:] = df_tmp.iloc[i:(i+1),rootdepth_indx].squeeze() # for 2 PFTs
    
    # Replace the default fates_allom_l2fr value with a value from the dataframe.
    l2frvar = fin.variables['fates_allom_l2fr'] 
    l2frvar[:] = df_tmp.iloc[i,l2fr_indx].squeeze() # for 1 PFT
    #rootlongvar[:] = df_tmp.iloc[i:(i+1),l2fr_indx].squeeze() # for 2 PFTs
    
    fnrtprofavar = fin.variables['fates_fnrt_prof_a']
    fnrtprofavar[:] = df_tmp.iloc[i,fnrtprofa_indx].squeeze() # for 1 PFT
    #rootlongvar[:] = df_tmp.iloc[i:(i+1),fnrtprofa_indx].squeeze() # for 2 PFTs
    
    fnrtprofbvar = fin.variables['fates_fnrt_prof_b']
    fnrtprofbvar[:] = df_tmp.iloc[i,fnrtprofb_indx].squeeze() # for 1 PFT
    #rootlongvar[:] = df_tmp.iloc[i:(i+1),fnrtprofb_indx].squeeze() # for 2 PFTs
    
    fin.close()
    

replacing file: param_file_fates_US-Rms_shrub_exp1_0001_c220630.nc
replacing file: param_file_fates_US-Rms_shrub_exp1_0002_c220630.nc
replacing file: param_file_fates_US-Rms_shrub_exp1_0003_c220630.nc
replacing file: param_file_fates_US-Rms_shrub_exp1_0004_c220630.nc
replacing file: param_file_fates_US-Rms_shrub_exp1_0005_c220630.nc
replacing file: param_file_fates_US-Rms_shrub_exp1_0006_c220630.nc
file does not yet exist: param_file_fates_US-Rms_shrub_exp1_0007_c220630.nc
file does not yet exist: param_file_fates_US-Rms_shrub_exp1_0008_c220630.nc
file does not yet exist: param_file_fates_US-Rms_shrub_exp1_0009_c220630.nc
file does not yet exist: param_file_fates_US-Rms_shrub_exp1_0010_c220630.nc


In [37]:
## Optional. Double check that the parameter values did in fact update properly
import xarray as xr

In [46]:
# Open a few of the new parameter files
ds1 = xr.open_mfdataset('param_file_fates_US-Rms_shrub_exp1_0001_c220630.nc', combine='by_coords')
ds5 = xr.open_mfdataset('param_file_fates_US-Rms_shrub_exp1_0005_c220630.nc', combine='by_coords')
ds10 = xr.open_mfdataset('param_file_fates_US-Rms_shrub_exp1_0010_c220630.nc', combine='by_coords')

In [47]:
# Check the frnt_prof_a values (should match df_tmp)
print(ds1.fates_fnrt_prof_a.values)
print(ds5.fates_fnrt_prof_a.values)
print(ds10.fates_fnrt_prof_a.values)

[5.]
[5.8]
[6.8]


In [48]:
# Check the zroot_max_z values (should match df_tmp)
print(ds1.fates_allom_zroot_max_z.values)
print(ds5.fates_allom_zroot_max_z.values)
print(ds10.fates_allom_zroot_max_z.values)

[1.]
[5.]
[10.]


In [49]:
# Check the wood_density values (should all be the same and be the set value in the original param file)
print(ds1.fates_wood_density.values)
print(ds5.fates_wood_density.values)
print(ds10.fates_wood_density.values)

[0.7]
[0.7]
[0.7]
